In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from torchvision.models.resnet import resnet18 as _resnet34
import torch
import torch.nn as nn
import torchvision as T

In [ ]:
def _make_resnet_backbone(resnet):
    pretrained = nn.Module()
    pretrained.conv0_0 = nn.Sequential(
    resnet.conv1, resnet.bn1, resnet.relu, resnet.layer1)
    pretrained.conv1_0 = resnet.layer2
    pretrained.conv2_0 = resnet.layer3
    pretrained.conv3_0 = resnet.layer4
    return pretrained

def resnet34(pretrained=True, **kwargs):
    model = _resnet34(pretrained=pretrained, **kwargs)
    return model


In [ ]:
class Upscale_Block(nn.Module):
    def __init__(self, input_channel, out):
        super().__init__()
        self.cv1 = nn.Conv2d(input_channel, out, 3, stride=(1, 1), padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out)
        self.cv2 = nn.Conv2d(out, out, 3, stride=(1, 1), padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out)
        self.relu = nn.ReLU(inplace=True)
        self.cv3 = nn.Conv2d(out, out, 3, stride=(1, 1), padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out)
        self.cv4 = nn.Conv2d(out, out, 3, stride=(1, 1), padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(out)

    def forward(self, x):
        out = self.cv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.cv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.cv3(out)
        out = self.bn3(out)
        out = self.relu(out)
        out = self.cv4(out)
        out = self.bn4(out)
        out = self.relu(out)

        return out

class Interpolate(nn.Module):
    def __init__(self, scale_factor, mode, align_corners=False):
        super(Interpolate, self).__init__()
        self.interp = nn.functional.interpolate
        self.scale_factor = scale_factor
        self.mode = mode
        self.align_corners = align_corners

    def forward(self, x):
        x = self.interp(
            x,
            scale_factor=self.scale_factor,
            mode=self.mode,
            align_corners=self.align_corners,
        )
        return x
    
class VGGBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, middle_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(middle_channels)
        self.conv2 = nn.Conv2d(middle_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        return out


In [ ]:
class DV3(nn.Module):
    def __init__(self, input_channels=3, **kwargs):
        super().__init__()
        nb_filter = [64, 128, 256, 512, 1024]
        model = resnet34()
        self.encoder = _make_resnet_backbone(model)
        #self.conv0_0 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv4_0 = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(512, 1024, 3, padding=1),
            nn.BatchNorm2d(1024),
            nn.MaxPool2d(2, 2),
        )
        self.final = nn.Conv2d(nb_filter[0], 1, kernel_size=1)
        self.up = Interpolate(scale_factor=2, mode="bilinear", align_corners=True)
        self.conv0_1 = VGGBlock(nb_filter[0] + nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_1 = VGGBlock(nb_filter[1] + nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_1 = VGGBlock(nb_filter[2] + nb_filter[3], nb_filter[2], nb_filter[2])
        self.conv3_1 = Upscale_Block(nb_filter[3] + nb_filter[4], nb_filter[3])
        self.conv0_2 = VGGBlock(nb_filter[0] * 2 + nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_2 = VGGBlock(nb_filter[1] * 2 + nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_2 = Upscale_Block(nb_filter[2] * 2 + nb_filter[3], nb_filter[2])
        self.conv0_3 = VGGBlock(nb_filter[0] * 3 + nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_3 = Upscale_Block(nb_filter[1] * 3 + nb_filter[2], nb_filter[1])
        self.conv0_4 = Upscale_Block(nb_filter[0] * 4 + nb_filter[1], nb_filter[0])

    def forward(self, input):
        # input_layer = self.conv0_0(input)
        x0_0 = self.encoder.conv0_0(input)
        x1_0 = self.encoder.conv1_0(x0_0)
        x2_0 = self.encoder.conv2_0(x1_0)
        x3_0 = self.encoder.conv3_0(x2_0)
        x4_0 = self.conv4_0(x3_0)
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], 1))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], 1))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], 1))
        out = self.up(x0_4)
        out = self.final(out)
        return out

class TransformerModel(nn.Module):
    def __init__(self, input_channels, output_channels, nhead=8, num_layers=6):
        super(TransformerModel, self).__init__()
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.nhead = nhead
        self.num_layers = num_layers

        model = _resnet34()
        self.encoder = _make_resnet_backbone(model)
        
        self.transformer_encoder_layers = nn.TransformerEncoderLayer(d_model=512, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layers, num_layers=num_layers)
        self.up = Interpolate(scale_factor=2, mode="bilinear", align_corners=True)
        
        # self.conv6 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1)
        self.conv6 = VGGBlock(512, 256, 128)
        # self.conv7 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1)
        self.conv7 = VGGBlock(128, 64, 32)
        # self.conv8 = nn.Conv2d(128, output_channels, kernel_size=3, stride=1, padding=1)
        self.conv8 = VGGBlock(32, 16, output_channels)

    def forward(self, x):
        x = self.encoder.conv0_0(x)
        x = self.encoder.conv1_0(x)
        x = self.encoder.conv2_0(x)
        x = self.encoder.conv3_0(x)
        
        x = x.permute(0, 2, 3, 1)
        b, h, w, c = x.shape
        x = x.reshape(b, h*w, c)
        
        x = self.transformer_encoder(x)
        
        x = x.reshape(b, h, w, c)
        x = x.permute(0, 3, 1, 2)
        
        x = F.relu(self.conv6(self.up(x)))
        x = F.relu(self.conv7(self.up(x)))
        x = self.conv8(self.up(x))
        
        return self.up(x)


In [ ]:
weights = TransformerModel(input_channels=3, output_channels=1)
#weights = torch.load("/kaggle/input/model4-weight1/model_4(5)_0.0019443425348819468.pth")
#lst = ["encoder"]
network = weights.cuda()
for name,param in network.named_parameters():
    param.requires_grad = True
#for name,param in weights.named_parameters():
    #print(name)
    #for i in lst:
        #if i in name:
            #param.requires_grad = False
    #print(name, '---->' ,param.requires_grad)

In [ ]:
from PIL import Image
import os, os.path
import cv2
import imageio
import torch.nn as nn
import glob
import time
import numpy as np
import scipy.ndimage
import math
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib.image import imread
%matplotlib inline

import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
from torchvision.io import read_image

In [ ]:
df_temp = pd.read_csv('../input/nyu-depth-v2/nyu_data/data/nyu2_train.csv', names=['image', 'label'])

In [ ]:
df_temp = df_temp.sample(frac = 1 , random_state = 79)

In [ ]:
df_train = df_temp[:42000]
df_val = df_temp[42000:]

In [ ]:
df_train = df_train.reset_index()
df_train.drop(['index'] , axis = 1 ,inplace = True)

df_val = df_val.reset_index()
df_val.drop(['index'] , axis = 1 ,inplace = True)

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, data_dir, transform=None, target_transform=None):
        '''
        The __init__ function is run once when instantiating the Dataset object
        '''
#         self.img_labels = pd.read_csv(annotations_file)
#         self.img_dir = img_dir
        self.root_dir = root_dir
        self.data_dir = data_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        '''
        The __len__ function returns the number of samples in dataset
        '''
        return len(self.data_dir)

    def __getitem__(self, idx):
        '''
        The __getitem__ function loads and returns a sample from the dataset at the given index idx
        '''
        img_path = os.path.join(self.root_dir, self.data_dir['image'][idx])
        image = Image.open(img_path)
        

        label_path = os.path.join(self.root_dir, self.data_dir['label'][idx])
        label = Image.open(label_path)
        
        seed = np.random.randint(2147483647) # make a seed with numpy generator 
        
        if self.transform is not None:
            random.seed(seed) # apply this seed to img tranfsorms
            torch.manual_seed(seed) # needed for torchvision 0.7
            image = self.transform(image)
        if self.target_transform is not None:
            random.seed(seed) # apply this seed to target tranfsorms
            torch.manual_seed(seed) # needed for  torchvision 0.7
            label = self.target_transform(label)
          
        return image, label

In [ ]:
root_dir = '../input/nyu-depth-v2/nyu_data'
train_dir = '/kaggle/working/train.csv'
val_dir = '/kaggle/working/val.csv'
transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply(transforms=[transforms.ColorJitter(brightness=.3, hue=.1)], p=0.25),
            transforms.RandomApply(transforms=[transforms.GaussianBlur(kernel_size=(11, 11), sigma=(0.1, 5))], p=0.5),
            transforms.Resize((512, 512)), # resize, the smaller edge will be matched.
            transforms.ToTensor(), # convert a PIL image or ndarray to tensor. 
])

val_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.Resize((512, 512)), # resize, the smaller edge will be matched.
            transforms.ToTensor(), # convert a PIL image or ndarray to tensor. 
])

target_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.Resize((512, 512)), # resize, the smaller edge will be ma
            transforms.ToTensor(), # convert a PIL image or ndarray to tensor. 
])

In [ ]:
train_dataset = CustomImageDataset(root_dir, df_train, transform=transform, target_transform=target_transform)

In [ ]:
validation_dataset = CustomImageDataset(root_dir, df_val, transform=val_transform, target_transform=target_transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=11, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=11, shuffle=True)

In [ ]:
#def vis(img, label):
 #   fig, axs = plt.subplots(1, 4, figsize=(15, 15))
  #  axs[0].imshow(img[0])
   # axs[1].imshow(img[1])
    #axs[2].imshow(transforms.ToPILImage()(img * 255), interpolation="bicubic")
    #axs[3].imshow(label)

#for v, (i, j) in enumerate(train_loader):
    #print('batch: ', v)
    #print(i.size(), torch.max(i), torch.min(i))
    #print(j.size(), torch.max(j), torch.min(j))
    #vis(i[0].squeeze(), j[0][0])
    #plt.show()

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(network.parameters(), lr = 0.0001)
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np

In [ ]:
min_valid_loss = np.inf
for epoch in range(20):
    
    total_loss = 0
    #total_correct = 0
    pbar = tqdm(train_loader)
    for batch in pbar:
        if torch.cuda.is_available():
            images,labels = batch
            images, labels = images.cuda(), labels.cuda()
        preds = network(images.float().cuda())
        t1 = nn.HuberLoss().cuda()
        #t1 = nn.BCEWithLogitsLoss().cuda()
        loss = t1(preds, labels.float().cuda())
        pbar.set_description(f'Loss -> {loss}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.to('cpu').item()
        #total_correct += get_num_correct(preds, labels)
        
    
    valid_loss = 0.0
    network.eval()     # Optional when not using Model Specific layer
    vbar = tqdm(validation_loader)
    with torch.no_grad():
        for vbatch in vbar:
            if torch.cuda.is_available():
                data, labels = vbatch
                data, labels = data.cuda(), labels.cuda()

            target = network(data.float()).cuda()
            t1 = nn.HuberLoss().cuda()
            loss = t1(target,labels).cuda()
            valid_loss += loss.to('cpu').item() 
            pbar.set_description(f'Validation Loss -> {loss}')
            
    print(f'Epoch {epoch+1} \t\t Training Loss: {total_loss} \t\t Validation Loss: {valid_loss/len(validation_loader)}')
    torch.save(network, f"model_5({epoch+1})_{valid_loss/len(validation_loader)}.pth")
        
    print(f'Epoch ==>> {epoch+1}  \t\t Loss ==>> {total_loss/len(train_loader)}')